In [2]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ed/b6/62345568cd07de5d9254fcf64d7e44aacbb6abde11ea953b3cb320e58d19/tensorflow-2.17.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.17.0 from https://files.pythonhosted.org/packages/66/03/5c447feceb72f5a38ac2aa79d306fa5b5772f982c2b480c1329c7e382900/tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2db

In [61]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from nilearn.connectome import ConnectivityMeasure

In [43]:
# Пути к данным
bnu_series_path = '../data/ts_cut/HCPex/bnu{}.npy'
bnu_labels_path = '../data/ts_cut/HCPex/bnu.csv'
ihb_series_path = '../data/ts_cut/HCPex/ihb.npy'
ihb_labels_path = '../data/ts_cut/HCPex/ihb.csv'

# Загрузка данных
X_bnu = np.concatenate([np.load(bnu_series_path.format(i)) for i in (1, 2)], axis=0)
Y_bnu = pd.read_csv(bnu_labels_path)
X_ihb = np.load(ihb_series_path)
Y_ihb = pd.read_csv(ihb_labels_path)

In [14]:
X_bnu_cutted = X_bnu[:, :120, :]  # Обрезаем временные шаги для BNU до 120
# Объединяем BNU и IHB данные
X = np.concatenate([X_bnu_cutted, X_ihb], axis=0)
Y = np.concatenate([Y_bnu.values, Y_ihb.values], axis=0)

print(f"Shape of X: {X.shape}, Shape of Y: {Y.shape}")

# Разбиваем данные на тренировочную и тестовую выборки
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=10)

# Уменьшение размерности с помощью PCA
def apply_pca(X, n_components=50):
    # Изменяем форму данных для PCA (складываем оси, чтобы использовать стандартные методы)
    n_samples, n_timepoints, n_regions = X.shape
    X_reshaped = X.reshape(n_samples * n_timepoints, n_regions)
    
    # Стандартизация данных
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_reshaped)
    
    # PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # Возвращаем данные к их исходной форме (n_samples, n_timepoints, n_components)
    X_pca_reshaped = X_pca.reshape(n_samples, n_timepoints, n_components)
    
    return X_pca_reshaped

# Применяем PCA к тренировочным и тестовым данным
n_components = 50  # Можно изменить это число, в зависимости от того, сколько признаков нужно оставить
X_train_pca = apply_pca(X_train, n_components=n_components)
X_val_pca = apply_pca(X_val, n_components=n_components)

print(f"Shape of X_train_pca: {X_train_pca.shape}, Shape of X_val_pca: {X_val_pca.shape}")

# Построение модели RNN (LSTM)
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Для бинарной классификации (открыты или закрыты глаза)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Параметры модели
input_shape = (X_train_pca.shape[1], X_train_pca.shape[2])  # (time_steps, features)

# Инициализация модели
model = build_lstm_model(input_shape)

# Обучение модели
history = model.fit(X_train_pca, Y_train, epochs=10, batch_size=16, validation_data=(X_val_pca, Y_val))

# Оценка модели
val_loss, val_accuracy = model.evaluate(X_val_pca, Y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

Shape of X: (162, 120, 419), Shape of Y: (162, 1)
Shape of X_train_pca: (137, 120, 50), Shape of X_val_pca: (25, 120, 50)
Epoch 1/10


C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 0.5670 - loss: 0.6788 - val_accuracy: 0.4800 - val_loss: 0.6889
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6747 - loss: 0.6037 - val_accuracy: 0.4800 - val_loss: 0.6759
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7595 - loss: 0.5493 - val_accuracy: 0.5200 - val_loss: 0.6758
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.7878 - loss: 0.4779 - val_accuracy: 0.5200 - val_loss: 0.6970
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.8987 - loss: 0.3675 - val_accuracy: 0.5600 - val_loss: 0.7405
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9517 - loss: 0.2676 - val_accuracy: 0.5600 - val_loss: 0.8019
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9930 - loss: 0.1412 - val_accuracy: 0.6400 - val_loss: 0.9018
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 0.0722 - val_accuracy: 0.6400 - val_loss: 1.1545
Epoch 9/10


In [56]:
def average_time_series(X, target_timepoints):
    n_samples, n_timepoints, n_features = X.shape
    factor = n_timepoints // target_timepoints  # Определяем шаг для усреднения
    X_avg = np.mean(X.reshape(n_samples, target_timepoints, factor, n_features), axis=2)
    return X_avg

# Приводим данные BNU к 120 временным шагам
X_bnu_avg = average_time_series(X_bnu, 120)


X = np.concatenate([X_bnu_avg, X_ihb], axis=0)
Y = np.concatenate([Y_bnu.values, Y_ihb.values], axis=0)

print(f"Shape of X: {X.shape}, Shape of Y: {Y.shape}")

# Разбиваем данные на тренировочную и тестовую выборки
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=10)

# Уменьшение размерности с помощью PCA
def apply_pca(X, n_components=50):
    # Изменяем форму данных для PCA (складываем оси, чтобы использовать стандартные методы)
    n_samples, n_timepoints, n_regions = X.shape
    X_reshaped = X.reshape(n_samples * n_timepoints, n_regions)
    
    # Стандартизация данных
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_reshaped)
    
    # PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # Возвращаем данные к их исходной форме (n_samples, n_timepoints, n_components)
    X_pca_reshaped = X_pca.reshape(n_samples, n_timepoints, n_components)
    
    return X_pca_reshaped

# Применяем PCA к тренировочным и тестовым данным
n_components = 300  # Можно изменить это число, в зависимости от того, сколько признаков нужно оставить
X_train_pca = apply_pca(X_train, n_components=n_components)
X_val_pca = apply_pca(X_val, n_components=n_components)

print(f"Shape of X_train_pca: {X_train_pca.shape}, Shape of X_val_pca: {X_val_pca.shape}")

# Построение модели RNN (LSTM)
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Для бинарной классификации (открыты или закрыты глаза)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Параметры модели
input_shape = (X_train_pca.shape[1], X_train_pca.shape[2])  # (time_steps, features)

# Инициализация модели
model = build_lstm_model(input_shape)

Shape of X: (162, 120, 419), Shape of Y: (162, 1)
Shape of X_train_pca: (137, 120, 300), Shape of X_val_pca: (25, 120, 300)


C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [57]:
# Обучение модели
history = model.fit(X_train_pca, Y_train, epochs=400, batch_size=30, validation_data=(X_val_pca, Y_val))

# Оценка модели
val_loss, val_accuracy = model.evaluate(X_val_pca, Y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')
model.save_weights(f'models/RNN{round(val_accuracy, 2)}.weights.h5')

Epoch 1/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 131ms/step - accuracy: 0.4716 - loss: 0.6964 - val_accuracy: 0.6000 - val_loss: 0.6863
Epoch 2/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.7062 - loss: 0.6339 - val_accuracy: 0.6800 - val_loss: 0.6774
Epoch 3/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7281 - loss: 0.5746 - val_accuracy: 0.6400 - val_loss: 0.6739
Epoch 4/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.7510 - loss: 0.5066 - val_accuracy: 0.6400 - val_loss: 0.6835
Epoch 5/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8178 - loss: 0.3965 - val_accuracy: 0.6400 - val_loss: 0.7268
Epoch 6/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9436 - loss: 0.2683 - val_accuracy: 0.6400 - val_loss: 0.7474
Epoch 7/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.1604 - val_accuracy: 0.6800 - val_loss: 0.8483
Epoch 8/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.1149 - val_accuracy: 0.6400 - val_loss

Epoch 52/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.6000 - val_loss: 2.1515
Epoch 53/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 0.6000 - val_loss: 2.1684
Epoch 54/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 0.0028 - val_accuracy: 0.6000 - val_loss: 2.1862
Epoch 55/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.6000 - val_loss: 2.2015
Epoch 56/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.6000 - val_loss: 2.2145
Epoch 57/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.6000 - val_loss: 2.2264
Epoch 58/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 0.0021 - val_accuracy: 0.6000 - val_loss: 2.2404
Epoch 59/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 0.6000 - v

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 8.5188e-04 - val_accuracy: 0.6000 - val_loss: 2.5785
Epoch 103/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 8.4231e-04 - val_accuracy: 0.6000 - val_loss: 2.5839
Epoch 104/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 6.9498e-04 - val_accuracy: 0.6000 - val_loss: 2.5882
Epoch 105/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 6.1251e-04 - val_accuracy: 0.6000 - val_loss: 2.5939
Epoch 106/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 7.6642e-04 - val_accuracy: 0.6000 - val_loss: 2.6003
Epoch 107/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 6.7680e-04 - val_accuracy: 0.6000 - val_loss: 2.6040
Epoch 108/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 6.9225e-04 - val_accuracy: 0.6000 - val_loss: 2.6080
Epoch 109/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 6.4707e-04 

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 3.6956e-04 - val_accuracy: 0.6400 - val_loss: 2.6509
Epoch 152/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 3.6711e-04 - val_accuracy: 0.6400 - val_loss: 2.6541
Epoch 153/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 3.2417e-04 - val_accuracy: 0.6400 - val_loss: 2.6569
Epoch 154/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 3.6399e-04 - val_accuracy: 0.6400 - val_loss: 2.6595
Epoch 155/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 3.8186e-04 - val_accuracy: 0.6400 - val_loss: 2.6620
Epoch 156/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 4.3444e-04 - val_accuracy: 0.6400 - val_loss: 2.6654
Epoch 157/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 3.3621e-04 - val_accuracy: 0.6400 - val_loss: 2.6682
Epoch 158/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 3.2078e-04 

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 2.4113e-04 - val_accuracy: 0.6400 - val_loss: 2.7932
Epoch 201/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 2.2327e-04 - val_accuracy: 0.6400 - val_loss: 2.7953
Epoch 202/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 2.1611e-04 - val_accuracy: 0.6400 - val_loss: 2.7971
Epoch 203/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 2.4264e-04 - val_accuracy: 0.6400 - val_loss: 2.7990
Epoch 204/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 2.1261e-04 - val_accuracy: 0.6400 - val_loss: 2.8005
Epoch 205/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 2.7341e-04 - val_accuracy: 0.6400 - val_loss: 2.8015
Epoch 206/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 2.2752e-04 - val_accuracy: 0.6400 - val_loss: 2.8030
Epoch 207/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 2.7689e-04 

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 1.7312e-04 - val_accuracy: 0.6400 - val_loss: 2.8851
Epoch 250/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 2.0434e-04 - val_accuracy: 0.6400 - val_loss: 2.8879
Epoch 251/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 1.9197e-04 - val_accuracy: 0.6400 - val_loss: 2.8910
Epoch 252/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 1.3946e-04 - val_accuracy: 0.6400 - val_loss: 2.8943
Epoch 253/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 1.0000 - loss: 1.6277e-04 - val_accuracy: 0.6400 - val_loss: 2.8968
Epoch 254/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 1.8298e-04 - val_accuracy: 0.6400 - val_loss: 2.8989
Epoch 255/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 1.4859e-04 - val_accuracy: 0.6400 - val_loss: 2.9014
Epoch 256/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 1.6195e-04 

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 1.1249e-04 - val_accuracy: 0.6400 - val_loss: 2.9517
Epoch 299/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 1.2086e-04 - val_accuracy: 0.6400 - val_loss: 2.9529
Epoch 300/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 1.0935e-04 - val_accuracy: 0.6400 - val_loss: 2.9533
Epoch 301/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 1.0000 - loss: 1.3114e-04 - val_accuracy: 0.6400 - val_loss: 2.9534
Epoch 302/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 1.3002e-04 - val_accuracy: 0.6400 - val_loss: 2.9540
Epoch 303/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 1.5165e-04 - val_accuracy: 0.6400 - val_loss: 2.9551
Epoch 304/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 1.3629e-04 - val_accuracy: 0.6400 - val_loss: 2.9568
Epoch 305/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 1.4262e-04 

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 8.1144e-05 - val_accuracy: 0.6400 - val_loss: 3.0043
Epoch 348/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 7.8469e-05 - val_accuracy: 0.6400 - val_loss: 3.0054
Epoch 349/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 1.4501e-04 - val_accuracy: 0.6400 - val_loss: 3.0070
Epoch 350/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 1.0496e-04 - val_accuracy: 0.6400 - val_loss: 3.0083
Epoch 351/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 7.3568e-05 - val_accuracy: 0.6400 - val_loss: 3.0092
Epoch 352/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 9.1162e-05 - val_accuracy: 0.6400 - val_loss: 3.0097
Epoch 353/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 9.3707e-05 - val_accuracy: 0.6400 - val_loss: 3.0090
Epoch 354/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 8.7405e-05 

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 5.9196e-05 - val_accuracy: 0.6400 - val_loss: 3.0076
Epoch 397/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 6.6447e-05 - val_accuracy: 0.6400 - val_loss: 3.0074
Epoch 398/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 8.8407e-05 - val_accuracy: 0.6400 - val_loss: 3.0083
Epoch 399/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 6.1422e-05 - val_accuracy: 0.6400 - val_loss: 3.0090
Epoch 400/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 8.7965e-05 - val_accuracy: 0.6400 - val_loss: 3.0090
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6400 - loss: 3.0090
Validation Loss: 3.0090386867523193, Validation Accuracy: 0.6399999856948853


In [30]:
X_train_pca.shape

(137, 120, 200)

In [55]:
# Функция для создания модели
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Бинарная классификация
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Функция для применения PCA и преобразования данных
def apply_pca(X, n_components):
    n_samples, n_timepoints, n_regions = X.shape
    X_reshaped = X.reshape(n_samples * n_timepoints, n_regions)
    
    # Стандартизация данных
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_reshaped)
    
    # PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # Возвращаем форму обратно для LSTM
    X_pca_reshaped = X_pca.reshape(n_samples, n_timepoints, n_components)
    
    return X_pca_reshaped

# Функция для кросс-валидации
def cross_validate_pca_lstm(X, Y, n_splits=5, pca_components=[50, 100], epochs_options=[10, 20], batch_size=32):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = []
    
    for n_components in pca_components:
        for epochs in epochs_options:
            fold_accuracy = []
            
            for train_index, val_index in kf.split(X):
                X_train, X_val = X[train_index], X[val_index]
                Y_train, Y_val = Y[train_index], Y[val_index]
                
                # Применение PCA к данным
                X_train_pca = apply_pca(X_train, n_components)
                X_val_pca = apply_pca(X_val, n_components)
                
                # Создаем и обучаем модель LSTM
                model = create_model(input_shape=(X_train_pca.shape[1], X_train_pca.shape[2]))
                
                # Callback для ранней остановки
                early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
                
                # Обучение модели
                history = model.fit(X_train_pca, Y_train, epochs=epochs, batch_size=batch_size,
                                    validation_data=(X_val_pca, Y_val), callbacks=[early_stopping], verbose=0)
                
                # Оценка на валидационном наборе
                val_loss, val_accuracy = model.evaluate(X_val_pca, Y_val, verbose=0)
                fold_accuracy.append(val_accuracy)
            
            # Средняя точность для текущих параметров
            mean_accuracy = np.mean(fold_accuracy)
            results.append({
                'n_components': n_components,
                'epochs': epochs,
                'mean_accuracy': mean_accuracy
            })
            print(f"n_components: {n_components}, epochs: {epochs}, mean_accuracy: {mean_accuracy}")
    
    return results

# Пример использования функции

# Пути к данным
bnu_series_path = '../data/ts_cut/HCPex/bnu{}.npy'
bnu_labels_path = '../data/ts_cut/HCPex/bnu.csv'
ihb_series_path = '../data/ts_cut/HCPex/ihb.npy'
ihb_labels_path = '../data/ts_cut/HCPex/ihb.csv'

# Загрузка данных
X_bnu = np.concatenate([np.load(bnu_series_path.format(i)) for i in (1, 2)], axis=0)
Y_bnu = pd.read_csv(bnu_labels_path)
X_ihb = np.load(ihb_series_path)
Y_ihb = pd.read_csv(ihb_labels_path)

# Приведение данных к общему виду (усреднение временных шагов или иные преобразования)
X_bnu_avg = average_time_series(X_bnu, 120)  # Среднее по временным шагам до 120
X = np.concatenate([X_bnu_avg, X_ihb], axis=0)
Y = np.concatenate([Y_bnu.values, Y_ihb.values], axis=0)

# Применение кросс-валидации для выбора гиперпараметров
results = cross_validate_pca_lstm(X, Y, pca_components=[150, 200, 250, 300], epochs_options=[30, 50, 100], batch_size=32)

# Вывод лучших параметров
best_result = max(results, key=lambda x: x['mean_accuracy'])
print(f"Best result: n_components={best_result['n_components']}, epochs={best_result['epochs']}, accuracy={best_result['mean_accuracy']}")


C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg

n_components: 150, epochs: 30, mean_accuracy: 0.6473484873771668


C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg

n_components: 150, epochs: 50, mean_accuracy: 0.6475378751754761


C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg

...

n_components: 300, epochs: 30, mean_accuracy: 0.672537875175476


C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg

n_components: 300, epochs: 50, mean_accuracy: 0.641287875175476


C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` arg

n_components: 300, epochs: 100, mean_accuracy: 0.635037875175476
Best result: n_components=250, epochs=30, accuracy=0.7164772748947144


In [59]:
def get_connectome(timeseries: np.ndarray,
                   conn_type: str = 'corr') -> np.ndarray:
    if conn_type == 'corr':
        conn = ConnectivityMeasure(kind='correlation', standardize=False).fit_transform(timeseries)
        conn[conn == 1] = 0.999999

        for i in conn:
            np.fill_diagonal(i, 0)

        conn = np.arctanh(conn)

    else:
        raise NotImplementedError

    return conn

In [64]:
X_bnu = get_connectome(X_bnu)
X_ihb = get_connectome(X_ihb)

# concat the train data
X = np.concatenate([X_bnu, X_ihb])
Y = np.concatenate([Y_bnu, Y_ihb])

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=10)



def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Для бинарной классификации (открыты или закрыты глаза)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Параметры модели
input_shape = (X_train_pca.shape[1], X_train_pca.shape[2])  # (time_steps, features)

# Инициализация модели
model = build_lstm_model(input_shape)

# Обучение модели
history = model.fit(X_train_pca, Y_train, epochs=500, batch_size=40, validation_data=(X_val_pca, Y_val))

Epoch 1/500


C:\Users\Public\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - accuracy: 0.5444 - loss: 0.6902 - val_accuracy: 0.5600 - val_loss: 0.6854
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7837 - loss: 0.6354 - val_accuracy: 0.6000 - val_loss: 0.6752
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8091 - loss: 0.5910 - val_accuracy: 0.6400 - val_loss: 0.6669
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8516 - loss: 0.5107 - val_accuracy: 0.6400 - val_loss: 0.6581
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8662 - loss: 0.4614 - val_accuracy: 0.6400 - val_loss: 0.6541
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.8783 - loss: 0.3937 - val_accuracy: 0.6400 - val_loss: 0.6567
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.9516 - loss: 0.2914 - val_accuracy: 0.6400 - val_loss: 0.6609
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 0.2023 - val_accuracy: 0.6800 - val_loss: 0.6544
Epo

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 0.6400 - val_loss: 1.1690
Epoch 53/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 0.0037 - val_accuracy: 0.6400 - val_loss: 1.1942
Epoch 54/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 0.6800 - val_loss: 1.2178
Epoch 55/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 0.6800 - val_loss: 1.2401
Epoch 56/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.6800 - val_loss: 1.2621
Epoch 57/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.6800 - val_loss: 1.2828
Epoch 58/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 0.0028 - val_accuracy: 0.6800 - val_loss: 1.3048
Epoch 59/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.6800 - val_loss: 1.32

Epoch 103/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 9.9609e-04 - val_accuracy: 0.6800 - val_loss: 1.8235
Epoch 104/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.6800 - val_loss: 1.8301
Epoch 105/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 9.7665e-04 - val_accuracy: 0.6800 - val_loss: 1.8362
Epoch 106/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 9.2788e-04 - val_accuracy: 0.6800 - val_loss: 1.8422
Epoch 107/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.6800 - val_loss: 1.8483
Epoch 108/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 9.2872e-04 - val_accuracy: 0.6800 - val_loss: 1.8541
Epoch 109/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 8.1141e-04 - val_accuracy: 0.6800 - val_loss: 1.8597
Epoch 110/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 0.001

...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 1.8962e-04 - val_accuracy: 0.6800 - val_loss: 2.4596
Epoch 300/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 1.8341e-04 - val_accuracy: 0.6800 - val_loss: 2.4614
Epoch 301/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 2.1959e-04 - val_accuracy: 0.6800 - val_loss: 2.4634
Epoch 302/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 1.9029e-04 - val_accuracy: 0.6800 - val_loss: 2.4655
Epoch 303/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 1.7168e-04 - val_accuracy: 0.6800 - val_loss: 2.4676
Epoch 304/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 1.5487e-04 - val_accuracy: 0.6800 - val_loss: 2.4696
Epoch 305/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 2.0041e-04 - val_accuracy: 0.6800 - val_loss: 2.4717
Epoch 306/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 1.5893e-04 

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 1.3105e-04 - val_accuracy: 0.6800 - val_loss: 2.5431
Epoch 349/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 1.5628e-04 - val_accuracy: 0.6800 - val_loss: 2.5448
Epoch 350/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 1.3735e-04 - val_accuracy: 0.6800 - val_loss: 2.5463
Epoch 351/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 1.3575e-04 - val_accuracy: 0.6800 - val_loss: 2.5479
Epoch 352/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 1.1868e-04 - val_accuracy: 0.6800 - val_loss: 2.5495
Epoch 353/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 1.6085e-04 - val_accuracy: 0.6800 - val_loss: 2.5512
Epoch 354/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 1.2467e-04 - val_accuracy: 0.6800 - val_loss: 2.5530
Epoch 355/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 1.1371e-04 

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 1.0000 - loss: 1.1021e-04 - val_accuracy: 0.6800 - val_loss: 2.6200
Epoch 398/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 9.1145e-05 - val_accuracy: 0.6800 - val_loss: 2.6215
Epoch 399/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 1.1586e-04 - val_accuracy: 0.6800 - val_loss: 2.6230
Epoch 400/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 9.6883e-05 - val_accuracy: 0.6800 - val_loss: 2.6245
Epoch 401/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 8.6490e-05 - val_accuracy: 0.6800 - val_loss: 2.6260
Epoch 402/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 9.5108e-05 - val_accuracy: 0.6800 - val_loss: 2.6274
Epoch 403/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 1.0997e-04 - val_accuracy: 0.6800 - val_loss: 2.6290
Epoch 404/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 8.1461e-05 

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 9.2243e-05 - val_accuracy: 0.6800 - val_loss: 2.6877
Epoch 447/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 8.4407e-05 - val_accuracy: 0.6800 - val_loss: 2.6890
Epoch 448/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 7.7668e-05 - val_accuracy: 0.6800 - val_loss: 2.6901
Epoch 449/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 1.0000 - loss: 9.8395e-05 - val_accuracy: 0.6800 - val_loss: 2.6913
Epoch 450/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 8.3341e-05 - val_accuracy: 0.6800 - val_loss: 2.6924
Epoch 451/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 1.0069e-04 - val_accuracy: 0.6800 - val_loss: 2.6936
Epoch 452/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 7.2422e-05 - val_accuracy: 0.6800 - val_loss: 2.6947
Epoch 453/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 7.6199e-05 

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 5.4928e-05 - val_accuracy: 0.6800 - val_loss: 2.7477
Epoch 496/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 8.4994e-05 - val_accuracy: 0.6800 - val_loss: 2.7488
Epoch 497/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 1.0000 - loss: 6.6724e-05 - val_accuracy: 0.6800 - val_loss: 2.7499
Epoch 498/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 1.0926e-04 - val_accuracy: 0.6800 - val_loss: 2.7512
Epoch 499/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 6.2375e-05 - val_accuracy: 0.6800 - val_loss: 2.7524
Epoch 500/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 1.0000 - loss: 9.0023e-05 - val_accuracy: 0.6800 - val_loss: 2.7535


In [65]:
val_loss, val_accuracy = model.evaluate(X_val_pca, Y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')
model.save_weights(f'models/RNN{round(val_accuracy, 2)}.weights.h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6800 - loss: 2.7535
Validation Loss: 2.7534990310668945, Validation Accuracy: 0.6800000071525574


In [72]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


test_series_path = './data/ts_cut/HCPex/predict.npy'


def load_test_data(test_series_path):
    X_test = np.load(test_series_path)
    print(f"Shape of X_test: {X_test.shape}")
    return X_test

def preprocess_data(X, target_timepoints=120):
    # Усреднение временных шагов, если необходимо
    def average_time_series(X, target_timepoints):
        n_samples, n_timepoints, n_features = X.shape
        factor = n_timepoints // target_timepoints
        X_avg = np.mean(X.reshape(n_samples, target_timepoints, factor, n_features), axis=2)
        return X_avg
    
    X_avg = average_time_series(X, target_timepoints)
    scaler = StandardScaler()
    n_samples, n_timepoints, n_features = X_avg.shape
    X_avg = X_avg.reshape(n_samples * n_timepoints, n_features)  # Преобразование для стандартизации
    X_scaled = scaler.fit_transform(X_avg)
    X_scaled = X_scaled.reshape(n_samples, n_timepoints, n_features)  # Обратное преобразование формы

    return X_scaled


def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def make_predictions(model, X_test):
    model.load_weights('models/RNN0.92.weights.h5')
    print("Weights loaded successfully.")
    predictions = model.predict(X_test)
    return predictions

def main():
    X_test = load_test_data(test_series_path)
    X_test_processed = preprocess_data(X_test, target_timepoints=300)
    model = create_model(input_shape=(X_test_processed.shape[1], X_test_processed.shape[2]))
    predictions = make_predictions(model, X_test_processed)
    print("Predictions:", predictions)

if __name__ == '__main__':
    main()

Shape of X_test: (142, 240, 419)


ValueError: cannot reshape array of size 14279520 into shape (142,300,0,419)